In [39]:
import warnings
warnings.filterwarnings("ignore")

In [40]:
from arch import arch_model
import pandas as pd
from pandas.tseries.offsets import MonthEnd, YearEnd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

### Returns Daily

In [41]:
daily_returns = pd.read_csv('Final_Returns_Data.csv', index_col=0)
daily_returns.index = pd.to_datetime(daily_returns.index)

### Returns Monthly

In [42]:
notional = pd.read_csv('Final_Notional_Data.csv', index_col=0, parse_dates=True)
# Step 1: Create a DataFrame of month-ends
month_ends = pd.date_range(start=notional.index.min(), end=notional.index.max(), freq='M')

# Step 2: For each month-end, find the closest date in df.index
closest_dates = []
for month_end in month_ends:
    closest_date = notional.index[np.abs((notional.index - month_end).days).argmin()]
    closest_dates.append(closest_date)

# Step 3: Extract rows at those closest dates
df_closest_to_month_end = notional.loc[sorted(set(closest_dates))]

returns_monthly = df_closest_to_month_end.pct_change().dropna()

# Deal with Athens
# Simple Linear interpolation
idx = returns_monthly["FTSE/Athens Notional Value"].idxmax()
pos = returns_monthly.index.get_loc(idx)
prior_date = returns_monthly.index[pos - 1]
returns_monthly.loc[prior_date]
following_date = returns_monthly.index[pos + 1]

# Add the average of the two surrounding values
returns_monthly["FTSE/Athens Notional Value"].loc[returns_monthly["FTSE/Athens Notional Value"].idxmax()] = (returns_monthly["FTSE/Athens Notional Value"].loc[following_date] + returns_monthly["FTSE/Athens Notional Value"].loc[prior_date]) / 2

returns_monthly.describe()

,DAX Notional Value,Amsterdam Index Notional Value,CAC40 Notional Value,PSI-20 Notional Value,IBEX-35 Notional Value,FTSE/Athens Notional Value,BEL-20 Notional Value,FTSE/MIB Notional Value,Gold Notional Value,Silver Notional Value
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.007121,0.006591,0.005938,0.002680,0.005943,0.001997,0.005134,0.005139,0.009129,0.010669
std,0.051665,0.047431,0.047944,0.050424,0.054821,0.091698,0.048334,0.058607,0.047736,0.092569
min,-0.173032,-0.199692,-0.168957,-0.202864,-0.215921,-0.305530,-0.208714,-0.217006,-0.180050,-0.279791
25%,-0.023088,-0.019074,-0.023549,-0.024552,-0.022831,-0.047848,-0.020714,-0.027160,-0.020996,-0.054097
50%,0.013294,0.012669,0.009803,0.005193,0.007164,0.008565,0.013739,0.009324,0.005240,0.000986
75%,0.037859,0.035262,0.036580,0.030809,0.035203,0.058153,0.034870,0.040573,0.039517,0.066292
max,0.169795,0.138109,0.202521,0.167766,0.258393,0.315495,0.210823,0.232054,0.140449,0.304222


In [43]:
returns_monthly.index = returns_monthly.index.to_period('M')
returns_monthly

,DAX Notional Value,Amsterdam Index Notional Value,CAC40 Notional Value,PSI-20 Notional Value,IBEX-35 Notional Value,FTSE/Athens Notional Value,BEL-20 Notional Value,FTSE/MIB Notional Value,Gold Notional Value,Silver Notional Value
Date,,,,,,,,,,
2004-05,-0.011564,-0.003418,0.012284,-0.039936,-0.013116,-0.027526,0.011813,-0.003884,0.017823,0.004100
2004-06,0.033241,0.021238,0.021609,0.028331,0.012084,-0.003642,0.033888,0.038000,-0.003806,-0.054212
2004-07,-0.044776,-0.043450,-0.024490,-0.036205,-0.014027,-0.015930,-0.002984,-0.020271,-0.004075,0.132930
2004-08,-0.027920,-0.011933,-0.014759,-0.006005,-0.004217,-0.007472,0.031859,-0.024127,0.049356,0.033269
2004-09,0.024865,-0.000079,0.009567,0.035819,0.019023,0.023637,0.051451,0.030095,0.019976,0.020465
...,...,...,...,...,...,...,...,...,...,...
2024-11,0.026995,0.008779,-0.018203,-0.019817,-0.000465,0.009610,0.002015,-0.017490,-0.030706,-0.061212
2024-12,0.009854,-0.007066,0.017121,-0.010945,-0.005875,0.062043,0.008392,0.019002,-0.011519,-0.057780
2025-01,0.087971,0.045883,0.075290,0.022646,0.066448,0.062129,0.012585,0.067697,0.069101,0.109015


In [44]:
idx = daily_returns["FTSE/Athens Notional Return"].idxmax()
position = daily_returns.index.get_loc(idx)
prior_day_retun = daily_returns["FTSE/Athens Notional Return"].iloc[position-1]
following_day_return = daily_returns["FTSE/Athens Notional Return"].iloc[position+1]
daily_returns["FTSE/Athens Notional Return"].iloc[position] = (prior_day_retun + following_day_return) / 2

In [ ]:
daily_returns_copy = daily_returns.copy()
daily_returns_copy.index = daily_returns_copy.index.to_period('M')

############################################################################################
# Simple Scaling Approach
############################################################################################
# Monthy Realized Variance
monthly_var = daily_returns_copy.groupby(daily_returns_copy.index).var()
monthly_var.columns = returns_monthly.columns

# Lag monthly_var by 1 period to use previous month's realized variance
monthly_var_lagged = monthly_var.shift(1)
monthly_var_lagged.columns = returns_monthly.columns
returns_monthly_scaled = returns_monthly.divide(monthly_var_lagged.loc[returns_monthly.index])
returns_monthly_scaled


############################################################################################
#Semi-variance Approach
############################################################################################
# Calculate the semi-variance
def semi_variance(returns):
    # Calculate the mean of the returns
    mean_return = returns.mean()
    # Calculate the semi-variance
    return ((returns[returns < mean_return] - mean_return) ** 2).mean()

semi_var_monthly = daily_returns_copy.groupby(daily_returns_copy.index).apply(semi_variance)
semi_var_monthly.columns = returns_monthly.columns
semi_var_monthly_lagged = semi_var_monthly.shift(1)

############################################################################################
# GARCH(1,1) Model
############################################################################################
# Fit GARCH(1,1) model to the daily returns
garch_models = {}
for column in daily_returns.columns:
    model = arch_model(daily_returns[column], vol='Garch', p=1, q=1)
    garch_models[column] = model.fit(disp="off")


conditional_variances = {}
for column, model in garch_models.items():
    conditional_variances[column] = model.conditional_volatility


conditional_variances_df = pd.DataFrame(conditional_variances, index=daily_returns.index)

# Calculate the monthly realized variance
monthly_realized_variance = (conditional_variances_df**2).groupby(conditional_variances_df.index.to_period('M')).mean()
monthly_realized_variance.columns = returns_monthly.columns

# Lag monthly_realized_variance by 1 period to use previous month's realized variance
monthly_realized_variance_lagged = monthly_realized_variance.shift(1)

# Scale the monthly returns using the GARCH model
returns_monthly_garch = returns_monthly.divide(monthly_realized_variance_lagged.loc[returns_monthly.index])

/opt/anaconda3/lib/python3.12/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


,DAX Notional Value,Amsterdam Index Notional Value,CAC40 Notional Value,PSI-20 Notional Value,IBEX-35 Notional Value,FTSE/Athens Notional Value,BEL-20 Notional Value,FTSE/MIB Notional Value,Gold Notional Value,Silver Notional Value
Date,,,,,,,,,,
2004-05,-0.940129,-0.043710,1.179428,-6.283735,-1.352623,-2.254735,0.231360,-0.474313,0.032243,0.160431
2004-06,2.374121,0.146647,1.821470,3.296348,1.010586,-0.258459,0.433165,3.645915,-0.003270,-2.105844
2004-07,-4.140319,-0.283552,-2.543419,-4.917415,-1.443976,-1.256975,-0.036546,-2.450998,-0.002978,6.168509
2004-08,-2.918407,-0.076164,-1.817868,-0.842349,-0.521684,-0.694497,0.398976,-3.283687,0.035046,1.558606
2004-09,2.295150,-0.000506,0.951820,5.147737,2.122547,1.907316,0.629890,3.392339,0.014108,1.149780
...,...,...,...,...,...,...,...,...,...,...
2024-11,3.193639,0.056480,-1.804778,-2.670635,-0.053551,0.820390,0.025143,-1.795952,-0.021672,-3.059968
2024-12,1.007741,-0.045226,1.730168,-1.027975,-0.578141,5.306220,0.105468,1.840421,-0.008118,-2.926987
2025-01,10.921420,0.296406,9.194500,3.159237,7.101251,5.436467,0.156282,7.506676,0.048708,5.846452


In [ ]:
#Normal Approach
returns_monthly_scaled

,DAX Notional Value,Amsterdam Index Notional Value,CAC40 Notional Value,PSI-20 Notional Value,IBEX-35 Notional Value,FTSE/Athens Notional Value,BEL-20 Notional Value,FTSE/MIB Notional Value,Gold Notional Value,Silver Notional Value
Date,,,,,,,,,,
2004-05,-74.471114,-52.507765,139.622858,-25409.461801,-228.523000,-236.884400,307.510778,-113.229417,86.477607,3.710222
2004-06,176.080177,132.321493,145.251812,347.698226,82.047696,-19.551713,296.676585,361.838795,-33.564209,-94.544877
2004-07,-431.044084,-834.879818,-397.487399,-1781.728542,-211.795366,-205.748700,-105.644013,-731.256668,-40.573625,280.705087
2004-08,-306.372591,-143.242673,-263.438018,-145.335140,-94.362710,-135.558702,520.400780,-756.942396,498.281950,82.076540
2004-09,273.443950,-0.759016,113.605730,1977.524383,365.258469,232.720261,1027.030242,636.363525,235.863101,123.194289
...,...,...,...,...,...,...,...,...,...,...
2024-11,698.693002,138.030291,-351.449002,-328.867143,-8.993619,126.273853,65.972933,-294.866028,-475.491276,-135.153619
2024-12,106.408520,-105.081149,201.130118,-116.720318,-60.917449,904.447380,91.605523,204.072682,-62.382296,-204.959391
2025-01,2790.749749,1594.273313,1805.862803,1016.512962,982.832109,1074.212894,447.703667,1241.341089,792.587761,339.163608


In [51]:
# Semi-variance scaling
returns_monthly_scaled_semi

,DAX Notional Value,Amsterdam Index Notional Value,CAC40 Notional Value,PSI-20 Notional Value,IBEX-35 Notional Value,FTSE/Athens Notional Value,BEL-20 Notional Value,FTSE/MIB Notional Value,Gold Notional Value,Silver Notional Value
Date,,,,,,,,,,
2004-05,-62.269758,-42.861363,143.327876,-21474.844688,-195.260891,-261.627677,523.756234,-160.647269,17.164773,3.248043
2004-06,127.171876,101.156800,150.047437,283.944143,78.015530,-17.138231,186.175607,358.797289,-34.770348,-72.703314
2004-07,-575.004471,-502.025205,-364.543921,-1816.093050,-213.111331,-224.610172,-108.416524,-775.690303,-39.317840,349.911074
2004-08,-659.023336,-148.967138,-270.065770,-145.076232,-76.333301,-135.866259,946.084391,-813.979665,652.276207,93.694425
2004-09,254.195747,-0.667208,89.759214,1343.686876,308.953584,204.100768,908.721504,494.991307,231.714769,70.102456
...,...,...,...,...,...,...,...,...,...,...
2024-11,805.406568,108.152974,-357.300087,-848.639251,-7.916542,133.303612,71.407000,-300.874928,-310.204317,-107.098180
2024-12,110.186956,-117.358275,167.234782,-84.505887,-41.537940,922.873887,59.786986,250.990495,-31.959679,-151.304027
2025-01,3839.321249,1179.674190,1574.729164,778.990298,751.060321,2694.356097,257.488872,1398.204929,805.056222,280.951542


In [53]:
returns_monthly_garch

,DAX Notional Value,Amsterdam Index Notional Value,CAC40 Notional Value,PSI-20 Notional Value,IBEX-35 Notional Value,FTSE/Athens Notional Value,BEL-20 Notional Value,FTSE/MIB Notional Value,Gold Notional Value,Silver Notional Value
Date,,,,,,,,,,
2004-05,-0.940129,-0.043710,1.179428,-6.283735,-1.352623,-2.254735,0.231360,-0.474313,0.032243,0.160431
2004-06,2.374121,0.146647,1.821470,3.296348,1.010586,-0.258459,0.433165,3.645915,-0.003270,-2.105844
2004-07,-4.140319,-0.283552,-2.543419,-4.917415,-1.443976,-1.256975,-0.036546,-2.450998,-0.002978,6.168509
2004-08,-2.918407,-0.076164,-1.817868,-0.842349,-0.521684,-0.694497,0.398976,-3.283687,0.035046,1.558606
2004-09,2.295150,-0.000506,0.951820,5.147737,2.122547,1.907316,0.629890,3.392339,0.014108,1.149780
...,...,...,...,...,...,...,...,...,...,...
2024-11,3.193639,0.056480,-1.804778,-2.670635,-0.053551,0.820390,0.025143,-1.795952,-0.021672,-3.059968
2024-12,1.007741,-0.045226,1.730168,-1.027975,-0.578141,5.306220,0.105468,1.840421,-0.008118,-2.926987
2025-01,10.921420,0.296406,9.194500,3.159237,7.101251,5.436467,0.156282,7.506676,0.048708,5.846452
